<h2><a href="https://leetcode.com/problems/evaluate-boolean-expression">1565. Evaluate Boolean Expression</a></h2><h3>Medium</h3><hr><p>Table <code>Variables</code>:</p>

<pre>
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| name          | varchar |
| value         | int     |
+---------------+---------+
In SQL, name is the primary key for this table.
This table contains the stored variables and their values.
</pre>

<p>&nbsp;</p>

<p>Table <code>Expressions</code>:</p>

<pre>
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| left_operand  | varchar |
| operator      | enum    |
| right_operand | varchar |
+---------------+---------+
In SQL, (left_operand, operator, right_operand) is the primary key for this table.
This table contains a boolean expression that should be evaluated.
operator is an enum that takes one of the values (&#39;&lt;&#39;, &#39;&gt;&#39;, &#39;=&#39;)
The values of left_operand and right_operand are guaranteed to be in the Variables table.
</pre>

<p>&nbsp;</p>

<p>Evaluate the boolean expressions in <code>Expressions</code> table.</p>

<p>Return the result table in <strong>any order</strong>.</p>

<p>The result format is in the following example.</p>

<p>&nbsp;</p>
<p><strong class="example">Example 1:</strong></p>

<pre>
<strong>Input:</strong> 
Variables table:
+------+-------+
| name | value |
+------+-------+
| x    | 66    |
| y    | 77    |
+------+-------+
Expressions table:
+--------------+----------+---------------+
| left_operand | operator | right_operand |
+--------------+----------+---------------+
| x            | &gt;        | y             |
| x            | &lt;        | y             |
| x            | =        | y             |
| y            | &gt;        | x             |
| y            | &lt;        | x             |
| x            | =        | x             |
+--------------+----------+---------------+
<strong>Output:</strong> 
+--------------+----------+---------------+-------+
| left_operand | operator | right_operand | value |
+--------------+----------+---------------+-------+
| x            | &gt;        | y             | false |
| x            | &lt;        | y             | true  |
| x            | =        | y             | false |
| y            | &gt;        | x             | true  |
| y            | &lt;        | x             | false |
| x            | =        | x             | true  |
+--------------+----------+---------------+-------+
<strong>Explanation:</strong> 
As shown, you need to find the value of each boolean expression in the table using the variables table.
</pre>

In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("EvaluateBooleanExpression").getOrCreate()

# Hardcoded Variables DataFrame
variables_data = [
    ("x", 66),
    ("y", 77)
]

variables_columns = ["name", "value"]

variables_df = spark.createDataFrame(variables_data, variables_columns)

# Hardcoded Expressions DataFrame
expressions_data = [
    ("x", ">", "y"),
    ("x", "<", "y"),
    ("x", "=", "y"),
    ("y", ">", "x"),
    ("y", "<", "x"),
    ("x", "=", "x")
]

expressions_columns = ["left_operand", "operator", "right_operand"]

expressions_df = spark.createDataFrame(expressions_data, expressions_columns)

# Show the DataFrames
print("Variables DataFrame:")
variables_df.show()

print("Expressions DataFrame:")
expressions_df.show()

25/02/19 08:44:08 WARN Utils: Your hostname, Nahils-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.103 instead (on interface en0)
25/02/19 08:44:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/19 08:44:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Variables DataFrame:


+----+-----+
|name|value|
+----+-----+
|   x|   66|
|   y|   77|
+----+-----+

Expressions DataFrame:
+------------+--------+-------------+
|left_operand|operator|right_operand|
+------------+--------+-------------+
|           x|       >|            y|
|           x|       <|            y|
|           x|       =|            y|
|           y|       >|            x|
|           y|       <|            x|
|           x|       =|            x|
+------------+--------+-------------+



In [2]:
# Create views for both DataFrames
variables_df.createOrReplaceTempView("Variables")
expressions_df.createOrReplaceTempView("Expressions")

In [7]:
result_df = spark.sql("""
-- Write your SQL query here
    SELECT e.left_operand,
           operator,
           e.right_operand,
           CASE
            WHEN operator = '<' THEN (v1.value < v2.value)
            WHEN operator = '>' THEN (v1.value > v2.value)
            WHEN operator = '=' THEN (v1.value = v2.value)
           END as result  
    FROM Expressions e
    INNER JOIN Variables v1 on v1.name = e.left_operand
    INNER JOIN Variables v2 on v2.name = e.right_operand
""")

result_df.show()

+------------+--------+-------------+------+
|left_operand|operator|right_operand|result|
+------------+--------+-------------+------+
|           y|       <|            x| false|
|           y|       >|            x|  true|
|           x|       =|            x|  true|
|           x|       =|            y| false|
|           x|       <|            y|  true|
|           x|       >|            y| false|
+------------+--------+-------------+------+



In [8]:
spark.stop()